# sentiment analysis on Twitter live data using Spark Streaming

Import sparkContext & StreamingContext from PySpark library.

In [ ]:
from pyspark import SparkContext

In [ ]:
from pyspark.streaming import StreamingContext

Create a sparkContext<br>
Create Spark Streaming Context using SC (spark context). parameter 10 is the batch interval. <br>
Every 10 second the analysis will be done.

In [ ]:
sc = SparkContext(master='local[2]',appName="sentiAnalysis")

In [ ]:
ssc = StreamingContext(sc,10)

In [ ]:
ssc.checkpoint("sentimentCheckpoint")

Connect to socket broker using ssc (spark streaming context)<br>
Host  (localhost) & port : 6782

In [ ]:
dstream = ssc.socketTextStream('127.0.0.1',6782)

Process the Stream and find sentiment on iphone in twitter<br>

1)Import the TextBlob Library which helps in finding the polarity value of a given text,<br>
Value ranges from -1 to +1 , -1 indicating negative sentiment and 0 indicating neutral and +1 indicating positive sentiment<br>
2)Get_sentiment() function helps in finding the sentiment of a given tweet using TextBlob library. <br>
3)Map transformation helps in calling the get_sentiment() and maps count to 1<br>
4)updateStateByKey helps in summing the overall count <br>
5)pprint helps in printing the analysis of sentiment<br>

In [ ]:
from textblob import TextBlob

In [ ]:
def get_sentiment(tweet):
    analyser=TextBlob(tweet)#.translate(to= 'en')
    senti=int(analyser.sentiment.polarity)
    if senti == 0:
        return "Neutral"
    elif senti > 0:
        return "Positive"
    elif senti < 0:
        return "Negative"
    else:
        return "Unknown"

In [ ]:
sentimentCount = dstream.map(lambda tweet: (get_sentiment(tweet),1))

aggregate_tags_count() is used to the overall sum of each intervals

In [ ]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [ ]:
#TotalSentiCount = sentimentCount.reduceByKey(lambda a,b:a+b)
TotalSentiCount = sentimentCount.updateStateByKey(aggregate_tags_count)

Print the final analysis: top trends on streaming twitter data

In [ ]:
TotalSentiCount.pprint()

### Starting the Spark Streaming:
Spark Streaming code we have written till now will not execute, untill we start the ssc.<br>
ssc.start() will start the spark streaming context. This is the Action for the whole code. <br>
Now it'll create the lineage & DAG & do the lazy evaluation & start running the whole sequesnce of code.


In [ ]:
ssc.start()

awaitTermination() is very important to stop the SSC.<br> 
When we kill this python process then this signal will be sent to awaitTermination() function.<br> 
it will finally stop the spark streaming job.

In [ ]:
ssc.awaitTermination()